# Auto AudioBooks



Preparing the environment


OCR for Optical Character Recognizition on Input PDF Books/Papers

In [1]:
!pip install pytesseract
!sudo apt install tesseract-ocr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (4,690 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like

Text to Speech for creating Audiobooks

In [2]:
!pip install gTTS
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 0s (421 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 122399 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [4]:
import pytesseract 
from gtts import gTTS
from IPython.display import Audio
from PIL import Image 
from pdf2image import convert_from_path 
import os


Utility to create DIR

In [5]:
def createDir(dirName):
  if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory " , dirName ,  " Created ")
  else:    
    print("Directory " , dirName ,  " already exists")


Utility to extract filename

In [6]:
def get_filename_without_extension(file_path):
    file_basename = os.path.basename(file_path)
    filename_without_extension = file_basename.split('.')[0]
    return filename_without_extension

In [7]:
def percentage(curr,total):
  return curr/total * 100;

Constants

In [8]:
IMAGES_DIR = "/content/Images/"
AUDIO_DIR = "/content/Audio/"
createDir(IMAGES_DIR)
createDir(AUDIO_DIR)

Directory  /content/Images/  Created 
Directory  /content/Audio/  Created 


Convert PDF to image snapshots

In [9]:
def pdf2Image(fn):
  file = get_filename_without_extension(fn)
  image_fn_dir= IMAGES_DIR + file
  createDir(image_fn_dir)
  print(fn);
  pages = convert_from_path(fn, 500)
  print(pages)
  filenames = [];
  
  # Counter to store images of each page of PDF to image 
  image_counter = 1
  total = len(pages)

  # Iterate through all the pages stored above 
  for page in pages: 
    
      # Declaring filename for each page of PDF as JPG 
      # For each page, filename will be: 
      # PDF page 1 -> page_1.jpg 
      # PDF page 2 -> page_2.jpg 
      # PDF page 3 -> page_3.jpg 
      # .... 
      # PDF page n -> page_n.jpg 
      filename = image_fn_dir+ "/page_"+str(image_counter)+".jpg"
      filenames.append(filename);
      # Save the image of the page in system 
      page.save(filename, 'JPEG') 
      
      print("Conversion - "+str(percentage(image_counter,total)) +" % work done yay!")

      # Increment the counter to update filename 
      image_counter = image_counter + 1
      
  return filenames

Convert Images to Text using pytesseract OCR

In [10]:
def convertImagesToText(filenames, ofilename):
  # Creating a text file to write the output 
  outfile = "out_text_"+ofilename+".txt"
  
  # Open the file in append mode so that  
  # All contents of all images are added to the same file 
  f = open(outfile, "a")
  for file in filenames:
    print(file)
    text = str(((pytesseract.image_to_string(Image.open(file)))))
    text = text.replace('-\n', '')     
    text = text + '\n'
    f.write(text)
  f.close()
  print(outfile + " prepared :) ")
  return outfile

Upload pdf to be converted to AudioBook ^_^

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Oxford Reading Tree Read With Biff, Chip, and Kipper_ First Stories_ Level 1_ Floppy Did This (Book) ( PDFDrive ).pdf to Oxford Reading Tree Read With Biff, Chip, and Kipper_ First Stories_ Level 1_ Floppy Did This (Book) ( PDFDrive ).pdf


Given all input pdfs, convert them to audio books

In [12]:
for fn in uploaded.keys():
  # predict images
  filename = get_filename_without_extension(fn)
  path = "/content/" + fn
  filenames = pdf2Image(path);
  outFile = convertImagesToText(filenames,filename);
  text = open(outFile).read()
  ta_tts = gTTS(text)
  ta_tts.save(AUDIO_DIR + filename + '.mp3')

Directory  /content/Images/Oxford Reading Tree Read With Biff, Chip, and Kipper_ First Stories_ Level 1_ Floppy Did This (Book) ( PDFDrive )  Created 
/content/Oxford Reading Tree Read With Biff, Chip, and Kipper_ First Stories_ Level 1_ Floppy Did This (Book) ( PDFDrive ).pdf
[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DDE9A0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DDEB50>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DDE3A0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DDEA60>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DA97F0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DA97C0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DA9820>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591x3250 at 0x7FD856DA9850>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2591

Preview first generated audiobook

In [13]:
preview = get_filename_without_extension(next(iter(uploaded)))

In [14]:
audio_file_preview = AUDIO_DIR+preview+".mp3"
print(audio_file_preview)

/content/Audio/Oxford Reading Tree Read With Biff, Chip, and Kipper_ First Stories_ Level 1_ Floppy Did This (Book) ( PDFDrive ).mp3


In [15]:
Audio(audio_file_preview, autoplay=True)